In [3]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install Unidecode

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.
Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)
     |████████████████████████████████| 245kB 2.7MB/s 


In [9]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [6]:
newfilepath = "embedding_wiki_100d_pt.txt"
filepath = "/tmp/ptwiki_20180420_100d.txt.bz2"
with open(newfilepath, 'wb') as new_file, bz2.BZ2File(filepath, 'rb') as file:
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(data)

#carregar
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(newfilepath, binary=False)

# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [14]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [10]:
def pre_processamento_texto(corpus):
    corpus_alt  = re.findall(r"\w+(?:'\w+')?|[^\w\s]",corpus)
    corpus_alt  = [t.lower() for t in corpus_alt] 
    lista_stops = stopwords.words("portuguese")
    corpus_alt  = [token for token in corpus_alt if token not in lista_stops]
    corpus_alt  = [re.sub(r"\d","", token) for token in corpus_alt]
    corpus_alt  = [token for token in corpus_alt if token not in string.punctuation]
    corpus_alt  = [unidecode(token) for token in corpus_alt]
    return corpus_alt



['excelente', 'produto', 'chegou', 'antes', 'prazo', 'indico', 'recomendo', 'produto', 'bom', 'pois', 'testei', 'aprovado']


In [15]:
###
# Pre_processamento de palavras das frases, removendo stopwords e acentos, pontuações
# e retorna tokens em minusculo
###

frase1_pre = pre_processamento_texto(frase1)
frase2_pre = pre_processamento_texto(frase2)
frase3_pre = pre_processamento_texto(frase3)
frase4_pre = pre_processamento_texto(frase4)



In [20]:
# Jaccard = Intercessão / União

def jaccard_method(f1, f2): 
    return len(set(f1).intersection(set(f2)))/len(set(f1).union(set(f2)))

# Frase1 e Frase2
# Frase1 e Frase3
# Frase2 e Frase3
# Frase1 e Frase4
print(jaccard_method(frase1_pre, frase2_pre))
print(jaccard_method(frase1_pre, frase3_pre))
print(jaccard_method(frase2_pre, frase3_pre))
print(jaccard_method(frase1_pre, frase4_pre))

0.10526315789473684
0.0
0.0
0.0


<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.

O par frase1 e frase2 possuí maior similaridade por conter dentro de seus tokens (palavras) semelhanças de uma com a outra, o que não ocorre com as demais palavras. Portanto, este resultado faz sentindo uma vez que a intercessão de palavras distintas é 0 entre as demais frases testadas.

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [24]:
bow = CountVectorizer()

vector = bow.fit_transform([' '.join(frase1_pre), 
                            ' '.join(frase2_pre),
                            ' '.join(frase3_pre),
                            ' '.join(frase4_pre)])

In [25]:
vector.shape

(4, 29)

In [26]:
pd.DataFrame(vector.todense())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,1,1,1,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,2,0,1,0,0,0,0,1,0
1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
frase1_bow = vector.todense()[0]
frase2_bow = vector.todense()[1]
frase3_bow = vector.todense()[2]
frase4_bow = vector.todense()[3]

In [28]:
print(distance.cosine(frase1_bow,frase2_bow))
print(distance.cosine(frase1_bow,frase3_bow))
print(distance.cosine(frase2_bow,frase3_bow))
print(distance.cosine(frase1_bow,frase4_bow))

0.8309691490542968
1.0
1.0
1.0


<b> Atividade </b>


4) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

As frases f1 - f3, f1 - f4, f2 - f3, tiveram resultados cosseno iguais a 1.0, o que nos mostra que sua similaridade é nula, podemos ver que ao abrir estas frases, as palavras usadas, são distintas. A menor distância cosseno foi da frase1 e frase2, exatamente como esperado, pois estas frases tem palavras em comum.

### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

5) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [29]:
np.mean(np.array([word_vectors[palavra] for palavra in frase1_pre]), axis=0).shape

(100,)

In [30]:
word_vectors["testei"]

array([ 0.7639,  0.3701, -0.8806, -0.2736, -1.1103,  0.4136,  0.147 ,
        0.6423, -0.1022,  0.5495, -0.1379,  0.8172, -0.4062,  0.0082,
       -0.3987,  0.6067,  0.5797, -0.3458, -0.3474, -0.3411, -0.2711,
        0.3904, -0.466 ,  0.1123, -0.0695, -0.1692, -0.3848, -0.4922,
       -0.7122,  0.402 ,  0.2955, -0.5608,  0.976 ,  0.5732, -0.0393,
       -0.0741,  0.4224,  0.319 ,  0.5793, -0.2527,  0.5185, -0.2147,
       -0.3736, -0.5574,  0.3233, -0.1902, -0.6013, -0.0891, -1.021 ,
        0.777 ,  0.3467,  0.046 ,  0.2413, -0.4255,  0.4669, -0.1585,
       -0.3398,  0.3088, -0.0472,  0.4295,  0.9259, -0.0159, -0.3963,
        0.1668,  0.0899, -0.0103,  0.1233,  0.9663, -0.3848,  0.3967,
       -0.5887,  0.8346,  0.2106, -0.6855, -0.0744, -0.4362, -0.8564,
        0.2475,  0.1882,  0.1293, -0.3085, -0.6424, -0.9031, -0.1352,
       -0.3293,  0.5941, -0.0351, -0.0781,  0.5484,  0.7644,  0.3971,
        0.0968,  0.2734,  0.3265,  0.2035,  0.2835,  0.173 , -0.3811,
       -0.1333, -0.2

In [31]:
np.array([word_vectors[palavra] for palavra in frase1_pre]).shape

(12, 100)

In [32]:
def get_embedding(f):
    return np.mean(np.array([word_vectors[palavra] for palavra in f if palavra in word_vectors.vocab]), axis=0)

In [33]:
frase1_emb = get_embedding(frase1_pre)
frase2_emb = get_embedding(frase2_pre)
frase3_emb = get_embedding(frase3_pre)
frase4_emb = get_embedding(frase4_pre)

In [34]:
print(distance.cosine(frase1_emb,frase2_emb))
print(distance.cosine(frase1_emb,frase3_emb))
print(distance.cosine(frase2_emb,frase3_emb))
print(distance.cosine(frase1_emb,frase4_emb))

0.15136468410491943
0.21427351236343384
0.24302351474761963
0.32684141397476196


<b>Atividade </b>

6) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

O resultado foi similar ao apresentado na questão anterior, as maiores distâncias foram das frases que nada tem em comum e a frase 1 e frase 2, que possuem suas similaridades, tiveram uma distância mais considerável. Vale lembrar que após aplicar o embedding as palavras pré-processadas, ganharam um peso extra que pode explicar alguns valores próximos de 0.20 que representaria um certo grau de similaridade.

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

7) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [35]:
print("Frase 1 e 2 ",word_vectors.wmdistance(frase1_pre, frase2_pre))
print("Frase 1 e 3 ",word_vectors.wmdistance(frase1_pre, frase3_pre))
print("Frase 2 e 3 ",word_vectors.wmdistance(frase2_pre, frase3_pre))
print("Frase 1 e 4 ",word_vectors.wmdistance(frase1_pre, frase4_pre))

Frase 1 e 2  3.0444450580164455
Frase 1 e 3  3.715277379918346
Frase 2 e 3  3.7981251887017256
Frase 1 e 4  4.125106107605237


<b> Atividade </b>

8) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Novamente, temos os valores se repetindo onde a frase com maior similaridade possui um valor menor de WMD que quanto maior seu índice maior é seu nível de dessimilaridade ou desigualdade entre as frases.

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

9) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [36]:
df = pd.read_csv("/tmp/imdb-reviews-pt-br.csv")

In [37]:
df["sentiment"].value_counts()

neg    24765
pos    24694
Name: sentiment, dtype: int64

## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

10) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [38]:
target = df["sentiment"].replace(["neg","pos"],[0,1])

In [39]:
target.shape

(49459,)

### Bag of Words (BOW)

<b> Atividade </b>

11) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [41]:
def pre_processamento_texto_return_str(corpus):
    
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]",corpus)
    
    lista_stopwords = stopwords.words("portuguese")
    corpus_alt = [t for t in corpus_alt if t not in lista_stopwords]
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt]
    
    corpus_alt_str = ' '.join(corpus_alt)
    
    return corpus_alt_str.lower()

In [42]:
df["text_pt_sem_stopwords"] = df["text_pt"].progress_apply(lambda x: pre_processamento_texto_return_str(x))

In [43]:
df.head()

,id,text_en,text_pt,sentiment,text_pt_sem_stopwords
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais vez sr costner arrumou filme tempo necess...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este exemplo motivo maioria filmes ação mesmos...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro tudo odeio raps imbecis poderiam agir...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem beatles puderam escrever músicas todos gos...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes fotos latão palavra apropriada verdade ...


<b> Atividade </b>

12) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [44]:
bow = CountVectorizer()

In [45]:
x_bow = bow.fit_transform(df["text_pt_sem_stopwords"])

In [46]:
vetor = bow.fit_transform([' '.join(frase1_pre),
                    ' '.join(frase2_pre),
                    ' '.join(frase3_pre),
                    ' '.join(frase4_pre)])

In [47]:
X_bow = bow.fit_transform(df["text_pt_sem_stopwords"])

In [48]:
X_bow.shape

(49459, 127624)

### Embedding

<b> Atividade </b>

13) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [49]:
def pre_processamento_texto_return_token(corpus):
    
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]",corpus)
    
    lista_stopwords = stopwords.words("portuguese")
    corpus_alt = [t for t in corpus_alt if t not in lista_stopwords]
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    
    corpus_alt = [re.sub(r'\d', '', t) for t in corpus_alt]
    
    
    return corpus_alt

In [50]:
df["text_pt_sem_stopwords_token"] = df["text_pt"].progress_apply(lambda x: pre_processamento_texto_return_token(x))

In [51]:
df.head()

,id,text_en,text_pt,sentiment,text_pt_sem_stopwords,text_pt_sem_stopwords_token
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais vez sr costner arrumou filme tempo necess...,"[Mais, vez, Sr, Costner, arrumou, filme, tempo..."
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este exemplo motivo maioria filmes ação mesmos...,"[Este, exemplo, motivo, maioria, filmes, ação,..."
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro tudo odeio raps imbecis poderiam agir...,"[Primeiro, tudo, odeio, raps, imbecis, poderia..."
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem beatles puderam escrever músicas todos gos...,"[Nem, Beatles, puderam, escrever, músicas, tod..."
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes fotos latão palavra apropriada verdade ...,"[Filmes, fotos, latão, palavra, apropriada, ve..."



<b> Atividade </b>

14) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [52]:
X_embedding = df["text_pt_sem_stopwords_token"].progress_apply(lambda x: get_embedding(x))

In [53]:
X_embedding.head()

0    [0.1722594, 0.20761016, -0.053872462, -0.22401...
1    [0.17371698, 0.187417, -0.064705655, -0.208970...
2    [0.23271771, 0.13545005, -0.0045176437, -0.179...
3    [0.26263848, 0.13604508, -0.07352549, -0.20097...
4    [0.21992578, 0.10650969, -0.09349515, -0.14721...
Name: text_pt_sem_stopwords_token, dtype: object

In [54]:
len(X_embedding)

49459

## Treinamento

### CountVectorizer


<b> Atividade </b>

15) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [55]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, target, random_state=123)

In [56]:
X_bow

<49459x127624 sparse matrix of type '<class 'numpy.int64'>'
	with 5267081 stored elements in Compressed Sparse Row format>

In [57]:
X_train_bow

<37094x127624 sparse matrix of type '<class 'numpy.int64'>'
	with 3944594 stored elements in Compressed Sparse Row format>

In [58]:
X_test_bow

<12365x127624 sparse matrix of type '<class 'numpy.int64'>'
	with 1322487 stored elements in Compressed Sparse Row format>


<b> Atividade </b>

16) Treine com uma regressão logística

In [59]:
modelo = LogisticRegression()

In [60]:
modelo.fit(X_train_bow, y_train_bow)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


<b> Atividade </b>

17) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [61]:
modelo.predict(X_test_bow)

array([1, 1, 1, ..., 0, 1, 1])

In [62]:
predicoes = modelo.predict(X_test_bow)

In [63]:
y_test_bow.shape

(12365,)

In [64]:
print(classification_report(y_test_bow, predicoes))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      6112
           1       0.87      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

18) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [65]:
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_embedding, target, random_state=123)

In [66]:
len(X_train_emb)

37094

In [67]:
len(y_train_emb)

37094


<b> Atividade </b>

19) Treine com uma regressão logística

In [68]:
modelo_regressao = LogisticRegression()

In [69]:
X_train_emb.shape

(37094,)

In [70]:
X_train_emb = pd.DataFrame([x for x in X_train_emb])

In [71]:
X_train_emb.shape

(37094, 100)

In [72]:
X_test_emb = pd.DataFrame([x for x in X_test_emb])

In [73]:
X_test_emb.shape

(12365, 100)

In [74]:
modelo.fit(X_train_emb, y_train_emb)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


<b> Atividade </b>

20) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas

In [75]:
predicoes = modelo.predict(X_test_emb)

In [76]:
print(classification_report(y_test_bow, predicoes))

              precision    recall  f1-score   support

           0       0.77      0.78      0.78      6112
           1       0.78      0.77      0.78      6253

    accuracy                           0.78     12365
   macro avg       0.78      0.78      0.78     12365
weighted avg       0.78      0.78      0.78     12365



In [77]:
modelo = modelo.predict(X_test_emb)

In [78]:
print(classification_report(y_test_emb, modelo))

              precision    recall  f1-score   support

           0       0.77      0.78      0.78      6112
           1       0.78      0.77      0.78      6253

    accuracy                           0.78     12365
   macro avg       0.78      0.78      0.78     12365
weighted avg       0.78      0.78      0.78     12365




<b> Atividade </b>

21) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

A estrutura do BOW permite se criar vetores com a relação de cada palavra e dar um "peso" para esta relação, o que pode ser uma explicação para a diferença de acertos entre este modelo e o tipo Embedding, que neste caso apresenta pouca eficiência por, talvez, faltar atributos de especificidade dentro do modelo para que ele possa ser mais acertivo.

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [79]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

In [80]:
texto_neg

'Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costners character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks hes better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutchers ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.'

In [81]:
texto_pos

'SPOILERS THROUGH: I really am in the minority on this one but I liked this movie. Its not a classic but its definitely involving and quite an adrenalin fueled ride. I definitely thought it was worth at least a 7 rating.Perhaps the reason I liked it is because I havent seen the original.Something tells me that with a movie like this its strongest fans will be the people who have not seen the original version and thus, have little to compare it to. This was not a masterpiece but I did get into it quite a lot and it actually made me want to see the original.There were a few things I liked about it. One was the casting of Kowalkski. Viggo Mortenson was superb and really brought a lot of charisma to the role. Since the bulk of the movie fell on his shoulders, he really needed to be excellent and he was. This was a great role for him.Another interesting thing about Vansishing Point was the fact that its made for television. I had no idea this was the case when watching it. It sure seemed li

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [83]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [84]:
analyzer = SentimentIntensityAnalyzer()

In [85]:
analyzer.polarity_scores(texto_neg)

{'compound': 0.3958, 'neg': 0.126, 'neu': 0.76, 'pos': 0.114}

In [86]:
analyzer.polarity_scores(texto_pos)

{'compound': 0.9969, 'neg': 0.084, 'neu': 0.737, 'pos': 0.179}

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [87]:
from textblob import TextBlob

In [88]:
sentence=TextBlob(texto_neg)
sentence.sentiment

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

In [89]:
sentence=TextBlob(texto_pos)
sentence.sentiment

Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

24) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [90]:
from afinn import Afinn

In [91]:
afinn = Afinn()

In [92]:
afinn.score(texto_neg)

8.0

In [93]:
afinn.score(texto_pos)

55.0

<b> Atividade </b>

25) Para você, qual ferramenta teve melhor comportamento?

Ao compararmos os três tipos de analises de sentimentos, podemos ver que o Vader possui um score mais detalhado de sentimentos positivos, negativos e neutros, por considerar cada sentença do conjunto de palavras e categoriza elas de acordo com o sentimento. 
Já o TextBlob trabalha com uma analise mais subjetiva, onde a interpretação de cada sentença deve ser um pouco mais precisa para poder ter uma maior acertiva, por ser uma metodologia baseada em polaridade e subjetividade. Por fim temos o Afinn, é uma abodagem mais direta, onde tem uma biblioteca com um dicionario de palavras e sua classificação como neutra, negativa ou positiva, este modelo não consegue cobrir textos com uma varição grande de palavras, que claro não estejam presentes em seu dicionário.

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [94]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 2.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=711061e6d2f63f990bb29ed12d213abf9b63c6862d30ad9de402a582b1a39860
  Stored in directory: /tmp/pip-ephem-wheel-cache-be5otrmw/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [99]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [100]:
nlp = spacy.load('en_core_web_md')

OSError: ignored

In [98]:
doc = nlp("This is some text that I am processing with Spacy")

NameError: ignored

In [ ]:
doc

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

In [ ]:
round(doc.vector[10], 6)